I tried doing a healpix thing, looks like it will take too long instead do ra dec limits but need to breakup based on healpix stuff

In [44]:
import numpy as np
import pandas as pd
import archive.database
import os
import timeit
import healpy as hp

In [45]:
print 'Query BLISS db image filepaths in delve-deep program...'
db = archive.database.Database(dbname='db-bliss')
db.connect()
conn = db.connection

Query BLISS db image filepaths in delve-deep program...


In [46]:
start_time = timeit.default_timer()
raset=np.linspace(180,190, num=100)
decset=np.linspace(-20,-10,num=100)
hpxlist=[]
for i,j in [(x,y) for x in raset for y in decset]:
    hpxlist.append(hp.ang2pix(4096,i,j,nest=False, lonlat=True))

In [47]:
def sphere_area(ra1,ra2,d1,d2):
    area=180/np.pi*(abs(ra1-ra2))*(np.sin(d1*np.pi/180)-np.sin(d2*np.pi/180))
    return(area)


def get_ralim(ra1,d1,d2,total_area):
    return total_area*np.pi/180/abs(np.sin(d1*np.pi/180)-np.sin(d2*np.pi/180))+ra1


print(get_ralim(170,-15,-25,100))

180.655296697


In [48]:
sphere_area(170,180.6,-15,-25)

99.481040289716887

In [49]:
np.sin(0*np.pi/180)-np.sin(1*np.pi/180)

-0.017452406437283512

In [50]:
hpxlist=np.unique(hpxlist)

In [51]:
query = """ select e.*,f.* 
            from se_archive_info f, exposure e 
            where f.expnum = e.expnum and
                    e.radeg is not NULL and e.decdeg is not NULL and 
                    abs(e.glat) > 10 and 
                    e.exptime > 1. and 
                    e.band in ('g','r','i','z') and 
                    f.filetype = 'fullcat'
                    and e.hpix in  (""" + ','.join((str(n) for n in hpxlist)) + """)
            order by f.expnum, f.ccdnum limit 10"""

In [6]:

df_filepaths = pd.read_sql_query(query, conn)
elapsed = timeit.default_timer() - start_time

In [7]:
print(elapsed)

142.002387047


In [11]:

ramin=180
ramax=190
decmin=-20
decmax=-10

In [52]:
ramin,ramax,decmin,decmax=170,180.6,-25,-15

query = """ select e.*,f.* 
            from se_archive_info f, exposure e 
            where f.expnum = e.expnum and
                    e.radeg is not NULL and e.decdeg is not NULL and 
                    abs(e.glat) > 10 and 
                    e.exptime > 1. and 
                    e.band in ('g','r','i','z') and 
                    f.filetype = 'fullcat'
                    and e.radeg > {0} and e.radeg < {1} and e.decdeg > {2} and e.decdeg < {3} 
            order by f.expnum, f.ccdnum limit 1000""".format(ramin,ramax,decmin,decmax)

" select e.*,f.* \n            from se_archive_info f, exposure e \n            where f.expnum = e.expnum and\n                    e.radeg is not NULL and e.decdeg is not NULL and \n                    abs(e.glat) > 10 and \n                    e.exptime > 1. and \n                    e.band in ('g','r','i','z') and \n                    f.filetype = 'fullcat'\n                    and e.radeg > 170 and e.radeg < 180.6 and e.decdeg > -25 and e.decdeg < -15 \n            order by f.expnum, f.ccdnum limit 1000"

In [54]:
start_time2 = timeit.default_timer()
df_filepaths2 = pd.read_sql_query(query2, conn)
elapsed2 = timeit.default_timer() - start_time2

In [55]:
print(elapsed2)

1.80170488358


In [20]:
df_filepaths2["radeg"]

0      180.615
1      180.615
2      180.615
3      180.615
4      180.615
5      180.615
6      180.615
7      180.615
8      180.615
9      180.615
10     180.615
11     180.615
12     180.615
13     180.615
14     180.615
15     180.615
16     180.615
17     180.615
18     180.615
19     180.615
20     180.615
21     180.615
22     180.615
23     180.615
24     180.615
25     180.615
26     180.615
27     180.615
28     180.615
29     180.615
        ...   
970    189.970
971    189.970
972    189.970
973    189.970
974    189.970
975    189.970
976    189.997
977    189.997
978    189.997
979    189.997
980    189.997
981    189.997
982    189.997
983    189.997
984    189.997
985    189.997
986    189.997
987    189.997
988    189.997
989    189.997
990    189.997
991    189.997
992    189.997
993    189.997
994    189.997
995    189.997
996    189.997
997    189.997
998    189.997
999    189.997
Name: radeg, dtype: float64

In [53]:
len(np.unique(hpxlist))

472146

In [34]:
df_filepaths.columns

Index([u'airmass', u'altitude', u'band', u'camshut', u'darktime', u'date_obs',
       u'decdeg', u'detsize', u'dimmsee', u'expnum', u'expreq', u'exptime',
       u'field', u'filename', u'filepath', u'filetype', u'filter', u'filtpos',
       u'ha', u'hpix', u'humidity', u'instrument', u'latitude', u'longitude',
       u'mjd_obs', u'nextend', u'nite', u'object', u'observatory', u'observer',
       u'obstype', u'pixscal1', u'pixscal2', u'pressure', u'proctype',
       u'prodtype', u'program', u'propid', u'proposer', u'radeg', u'radesys',
       u'tdecdeg', u'teldec', u'telequin', u'telescope', u'telfocus', u'telra',
       u'telstat', u'tiling', u'timesys', u'time_obs', u'tradeg', u'winddir',
       u'windspd', u'zd', u'glon', u'glat', u'attnum', u'band', u'ccdnum',
       u'compression', u'expnum', u'filename', u'filetype', u'path', u'reqnum',
       u'status'],
      dtype='object')

In [19]:
hpxlist=tuple([45042308, 45042340])

In [36]:
print(np.unique(df_filepaths["hpix"]),np.unique(df_filepaths["radeg"]),np.unique(df_filepaths["decdeg"]))

(array([45042308, 45042340]), array([ 239.143,  239.862]), array([ 33.5443,  33.5451]))


In [1]:
import healpy as hp

In [11]:
hp.pix2ang(4096,45042308,lonlat=True,nest=False)

(239.150390625, 33.543615977327818)

In [26]:
df_filepaths["decdeg"]

0    33.5451
1    33.5451
2    33.5451
3    33.5451
4    33.5451
5    33.5451
6    33.5451
7    33.5451
8    33.5451
9    33.5451
Name: decdeg, dtype: float64

In [15]:
#!/usr/bin/env python

# Initial setup...
import numpy as np
import pandas as pd
import archive.database
import os

# Query BLISS db image filepaths in delve-deep program...
print 'Query BLISS db image filepaths in delve-deep program...'
db = archive.database.Database(dbname='db-bliss')
db.connect()
conn = db.connection


ramin,ramax,decmin,decmax=170,180.6,-25,-15

print("Ra lim:", ramin,"-",ramax,)
print("dec lim:", decmin,"-",ramax,)

query = """ select f.expnum, f.ccdnum, e.band, f.path||'/'||f.filename||f.compression as filepath
            from se_archive_info f, exposure e 
            where f.expnum = e.expnum and
                    e.radeg is not NULL and e.decdeg is not NULL and 
                    abs(e.glat) > 10 and 
                    e.exptime > 1. and 
                    e.band in ('g','r','i','z') and 
                    f.filetype = 'fullcat'
                    and e.radeg > {0} and e.radeg < {1} and e.decdeg > {2} and e.decdeg < {3} 
            order by f.expnum, f.ccdnum""".format(ramin,ramax,decmin,decmax)


df_filepaths = pd.read_sql_query(query, conn)

conn.close()



Query BLISS db image filepaths in delve-deep program...
('Ra lim:', 170, '-', 180.6)
('dec lim:', -25, '-', 180.6)


In [16]:
df_filepaths.shape

(58361, 4)

In [17]:
# Make all columns uppercase...
df_filepaths.columns = [x.upper() for x in df_filepaths.columns]

# Output df_filepaths...
#df_filepaths.to_csv('DELVE_Calib_filepaths_fnal.csv',index=False)


# Query BLISS db exposures in delve-deep program...
print 'Query BLISS db exposures in delve-deep program...'
db = archive.database.Database(dbname='db-bliss')
db.connect()
conn = db.connection
query = \
    """select e.expnum,
            e.radeg as EXPRA, e.decdeg as EXPDEC, e.exptime, e.airmass,
            e.band, e.nite, e.mjd_obs,
            REPLACE(e.field,   ',', ' ') as field,
            REPLACE(e.object,  ',', ' ') as object,
            REPLACE(e.program, ',', ' ') as program
       from exposure e
       where 
            e.radeg is not NULL and e.decdeg is not NULL and 
            abs(e.glat) > 10 and 
            e.exptime > 1. and 
            e.band in ('g','r','i','z') 
            and e.radeg > {0} and e.radeg < {1} and e.decdeg > {2} and e.decdeg < {3} 
        order by e.expnum""".format(ramin,ramax,decmin,decmax)

df_expinfo = pd.read_sql_query(query, conn)

conn.close()

Query BLISS db exposures in delve-deep program...


In [18]:
df_expinfo.shape

(1037, 11)

In [19]:
# Make all columns uppercase...
df_expinfo.columns = [x.upper() for x in df_expinfo.columns]

# It turns out that the column 'expnum' is duplicated in the above query.
# To remove all-but-the-first of any and all duplicated columns
# (see:  https://stackoverflow.com/questions/14984119/python-pandas-remove-duplicate-columns ):
df_expinfo = df_expinfo.loc[:,~df_expinfo.columns.duplicated()]

# Output df_expinfo...
#df_expinfo.to_csv('DELVE_Calib_expinfo_fnal.csv',index=False)


# Query BLISS db images in delve-deep program...
print 'Query BLISS db images in delve-deep program...'
db = archive.database.Database(dbname='db-bliss')
db.connect()
conn = db.connection

query = \
    """select i.expnum, i.ccdnum, i.band, 
              i.rac1, i.decc1, i.rac2, i.decc2, i.rac3, i.decc3, i.rac4, i.decc4,
              i.ra_cent, i.dec_cent, i.racmax, i.racmin, i.crossra0
       from image i, exposure e
       where i.expnum = e.expnum and 
             e.radeg > {0} and e.radeg < {1} and e.decdeg > {2} and e.decdeg < {3} and 
             i.band in ('g','r','i','z')
       order by i.expnum, i.ccdnum""".format(ramin,ramax,decmin,decmax)

df_imginfo = pd.read_sql_query(query, conn)
conn.close()


Query BLISS db images in delve-deep program...


In [20]:
df_imginfo.shape

(58361, 16)

In [21]:
df_imginfo.columns

Index([u'expnum', u'ccdnum', u'band', u'rac1', u'decc1', u'rac2', u'decc2',
       u'rac3', u'decc3', u'rac4', u'decc4', u'ra_cent', u'dec_cent',
       u'racmax', u'racmin', u'crossra0'],
      dtype='object')

In [22]:
df_filepaths.columns

Index([u'EXPNUM', u'CCDNUM', u'BAND', u'FILEPATH'], dtype='object')

In [23]:
df_filepaths.columns.duplicated()

array([False, False, False, False], dtype=bool)

In [24]:
print(df_filepaths.loc[2])

EXPNUM                                                 175613
CCDNUM                                                      3
BAND                                                        z
FILEPATH    /data/des50.b/data/BLISS/175600/175613/D001756...
Name: 2, dtype: object


In [26]:
df_imginfo.columns = [x.upper() for x in df_imginfo.columns]
print 'Merge df_imginfo with df_filepaths...'
df_imgfileinfo = pd.merge(df_imginfo, df_filepaths, \
                              on=['EXPNUM','CCDNUM'], \
                              how='inner', \
                              suffixes=('','_y')).reset_index(drop=True)

Merge df_imginfo with df_filepaths...


In [29]:
df_imgfileinfo

,EXPNUM,CCDNUM,BAND,RAC1,DECC1,RAC2,DECC2,RAC3,DECC3,RAC4,DECC4,RA_CENT,DEC_CENT,RACMAX,RACMIN,CROSSRA0,BAND_y,FILEPATH
0,175613,1,z,180.002700,-20.316198,180.001805,-20.465406,180.321065,-20.466139,180.321566,-20.317069,180.161722,-20.391235,180.321566,180.001805,N,z,/data/des50.b/data/BLISS/175600/175613/D001756...
1,175613,2,z,180.333604,-20.317164,180.332829,-20.466232,180.652299,-20.467223,180.652578,-20.318156,180.492791,-20.392185,180.652578,180.332829,N,z,/data/des50.b/data/BLISS/175600/175613/D001756...
2,175613,3,z,180.664764,-20.317478,180.664759,-20.466550,180.984026,-20.466784,180.983639,-20.317574,180.824283,-20.392121,180.984026,180.664759,N,z,/data/des50.b/data/BLISS/175600/175613/D001756...
3,175613,4,z,179.836175,-20.479562,179.835002,-20.628821,180.154484,-20.629760,180.155292,-20.480565,179.995145,-20.554691,180.155292,179.835002,N,z,/data/des50.b/data/BLISS/175600/175613/D001756...
4,175613,5,z,180.167372,-20.480539,180.166657,-20.629726,180.486630,-20.630189,180.486828,-20.480998,180.326800,-20.555328,180.486828,180.166657,N,z,/data/des50.b/data/BLISS/175600/175613/D001756...
5,175613,6,z,180.498983,-20.481052,180.498688,-20.630241,180.818668,-20.630701,180.818450,-20.481505,180.658691,-20.555842,180.818668,180.498688,N,z,/data/des50.b/data/BLISS/175600/175613/D001756...
6,175613,7,z,180.830402,-20.481505,180.830704,-20.630700,181.150191,-20.630709,181.149524,-20.481440,180.990221,-20.556100,181.150191,180.830402,N,z,/data/des50.b/data/BLISS/175600/175613/D001756...
7,175613,8,z,179.669351,-20.643061,179.668095,-20.792429,179.987641,-20.793521,179.988591,-20.644139,179.828308,-20.718308,179.988591,179.668095,N,z,/data/des50.b/data/BLISS/175600/175613/D001756...
8,175613,9,z,180.000709,-20.644024,179.999617,-20.793405,180.319898,-20.794339,180.320518,-20.644905,180.160071,-20.719141,180.320518,179.999617,N,z,/data/des50.b/data/BLISS/175600/175613/D001756...
9,175613,10,z,180.332537,-20.644938,180.332048,-20.794376,180.652670,-20.794803,180.652639,-20.645369,180.492434,-20.719829,180.652670,180.332048,N,z,/data/des50.b/data/BLISS/175600/175613/D001756...


In [78]:
# Make all columns uppercase...
df_imginfo.columns = [x.upper() for x in df_imginfo.columns]

# Output df_imginfo...
#df_imginfo.to_csv('DELVE_Calib_imginfo_fnal.csv',index=False)


# Merge df_imginfo with df_filepaths...
print 'Merge df_imginfo with df_filepaths...'
df_imgfileinfo = pd.merge(df_imginfo, df_filepaths, \
                              on=['EXPNUM','CCDNUM'], \
                              how='inner', \
                              suffixes=('','_y')).reset_index(drop=True)

# Remove any redundant columns...
to_drop = [colname for colname in df_imgfileinfo if colname.endswith('_y')]
df_imgfileinfo.drop(to_drop, axis=1, inplace=True)

# Output df_imgfileinfo...
df_imgfileinfo.to_csv('DELVE_Calib_imgfileinfo_fnal.csv',index=False)


# Merge df_expinfo with df_imgfileinfo...
print 'Merge df_expinfo with df_imgfileinfo...'
df_expimgfileinfo = pd.merge(df_expinfo, df_imgfileinfo, \
                                 on=['EXPNUM'], \
                                 how='inner', \
                                 suffixes=('','_y')).reset_index(drop=True)

# Remove any redundant columns...
to_drop = [colname for colname in df_expimgfileinfo if colname.endswith('_y')]
df_expimgfileinfo.drop(to_drop, axis=1, inplace=True)

# Output df_expimginfo...
#df_expimgfileinfo.to_csv('DELVE_Calib_expimgfileinfo_fnal.csv',index=False)



Merge df_imginfo with df_filepaths...


ValueError: Buffer has wrong number of dimensions (expected 1, got 2)

In [ ]:
# If it does not alreayd exist, create a subdiretory called Downloads...
if not os.path.exists('Downloads'):
    os.makedirs('Downloads')

# Loop through df_expimgfileinfo, copying the delve-deep 
#  files into the Downloads sub-directory...
# (Could do symbolic links, but better to copy, just to 
#  be safe and not accidentally delete or modify the 
#  original files...)
for index, row in df_expimgfileinfo.iterrows():
    print row['EXPNUM'], row['FILEPATH']
    if os.path.exists(row['FILEPATH']):
        cmd="""cp -p %s ./Downloads/""" % (row['FILEPATH'])
        os.system(cmd)
    else:
        """WARNING:  %s does not exist!  Skipping...""" % (row['FILEPATH'])


print 'Finis!'
